<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [104]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

#### Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/train'
validation_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/test'

In [5]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
validation_normal_dir = os.path.join(validation_dir,'NORMAL')
validation_pneumonia_dir = os.path.join(validation_dir,'PNEUMONIA')

train_pneumonia_dir

'/content/drive/My Drive/Senior Project/xray_dataset_covid19/train/PNEUMONIA'

#### สร้างตารางเก็บชื่อภาพ และชื่อโรค 
ข้อมูลในไฟล์ train ทั้ง normal, pneunomia และ lung canser มารวมอยู่ในไฟล์ all_train แล้วลองนำมาทำ data augment

In [6]:
#ที่เก็บรูปภาพแยกเป็น 3 คลาส สำหรับ สร้างตาราง ชื่อภาพ+ชื่อคลาส 

train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
train_ca_dir = os.path.join(train_dir,'CA')

In [7]:
all_train = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/all_train'    #ที่เก็บรูปภาพทั้งหมด ตอนนี้มี 3 คลาส

สร้าง data dict ของรูป

In [8]:
normal = []
for i in range(len(os.listdir(train_normal_dir))):
    img = os.listdir(train_normal_dir)[i]
    normal.append(img)
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'Type1': "Normal" ,
                                       'Type2': 0                      })

In [9]:
pneumonia = []
for i in range(len(os.listdir(train_pneumonia_dir))):
    img = os.listdir(train_pneumonia_dir)[i]
    pneumonia.append(img)
tpneumonia = pd.DataFrame({'ImageName':[pneumonia][0],
                                       'Type1': "Pneumonia" ,
                                       'Type2': 1                        })

In [10]:
tlungcancer = []
for i in range(len(os.listdir(train_ca_dir))):
    img = os.listdir(train_ca_dir)[i]
    tlungcancer.append(img)
tlungcancer = pd.DataFrame({'ImageName':[tlungcancer][0],
                                       'Type1': "Lungcancer" ,
                                       'Type2': 2                        })

In [11]:
data_train2 = pd.concat([tnormal, tpneumonia])
data_train2.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [13]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcancer])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


# 2 class
*   #1 output 1 มิติ  ความน่าจะเป็น นำมาแปลง 0,1
*   #2 output 2 มิติ  ความน่าจะเป็นที่จะเป็นคลาส 0 หรือ 1 



In [62]:
train_img = []
for ImageName in tqdm(data_train2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)

100%|██████████| 148/148 [00:06<00:00, 22.23it/s]


## 1) output 1 มิติ  ค่า predict คือ ความน่าจะเป็น 1 ค่า 
ถ้า > 0.5 แปลงคลาส 1

ถ้า < 0.5 แปลงคลาส 0

####แบ่งข้อมูล2_1

In [63]:
data2_1 = np.array(train_img, dtype="float32") / 255.0

In [64]:
labels2_1 = np.array(data_train2[['Type1']] )  

le = LabelEncoder()                                                                      # จะแปลงชื่อ class เป็นตัวเลข
labels2_1 = le.fit_transform(labels2_1)                                                  # dimention ของ Y จะเป็น 1 คือ (37,) ต้องเปลี่ยนเป็น (37,1) ก่อน ถึงจะรัน report ได้

labels2_1.shape = (len(labels2_1), 1)                                                   #ลองดูว่าถ้าไม่แปลงเป็น 2 มิติ จะรันได้มั้ย สรุป ต้องเปลี่ยนเป็น 2 มิติอยู่ดี เพราะ predict ที่รันออกมาเป็น 2 มิติ

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [65]:
le.classes_                 #คือ ['Normal', 'Pneumonia']

array(['Normal', 'Pneumonia'], dtype=object)

In [66]:
(trainX2_1, testX2_1, trainY2_1, testY2_1) = train_test_split(data2_1, labels2_1,
										   					test_size=0.25, random_state=30)

#### train2_1

In [67]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [68]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [69]:
base_model2_1 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

74850304/74836368 [==============================] - 1s 0us/step


In [70]:
base_model2_1.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2_1.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)   #  unit = 1 คือ มิติของ output เป็น 1 มีค่าความน่าจะเป็นค่าเดียว
model2_1 = tf.keras.models.Model(inputs=base_model2_1.input, outputs=prediction_layer)

In [71]:
model2_1.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [73]:
# train the network

H2_1 = model2_1.fit(
	x=aug.flow(trainX2_1, trainY2_1, batch_size=BS),
	validation_data=(testX2_1, testY2_1),
	steps_per_epoch=len(trainX2_1) // BS,
	epochs=12)

Epoch 1/12
13/13 [==============================] - 27s 2s/step - loss: 0.7971 - accuracy: 0.4660 - val_loss: 0.7096 - val_accuracy: 0.4595
Epoch 2/12
13/13 [==============================] - 29s 2s/step - loss: 0.7974 - accuracy: 0.4854 - val_loss: 0.6870 - val_accuracy: 0.5676
Epoch 3/12
13/13 [==============================] - 29s 2s/step - loss: 0.7993 - accuracy: 0.4369 - val_loss: 0.6630 - val_accuracy: 0.5676
Epoch 4/12
13/13 [==============================] - 29s 2s/step - loss: 0.7256 - accuracy: 0.4660 - val_loss: 0.6413 - val_accuracy: 0.5946
Epoch 5/12
13/13 [==============================] - 29s 2s/step - loss: 0.7185 - accuracy: 0.5049 - val_loss: 0.6204 - val_accuracy: 0.6757
Epoch 6/12
13/13 [==============================] - 28s 2s/step - loss: 0.7255 - accuracy: 0.5146 - val_loss: 0.6007 - val_accuracy: 0.6757
Epoch 7/12
13/13 [==============================] - 28s 2s/step - loss: 0.6815 - accuracy: 0.5728 - val_loss: 0.5829 - val_accuracy: 0.7027
Epoch 8/12
13/13 [==

#### test2_1

In [80]:
predictions2_1 = model2_1.predict(testX2_1, batch_size=BS)

In [81]:
predictions2_1[:5]

array([[0.33652294],
       [0.6935322 ],
       [0.76863503],
       [0.743651  ],
       [0.6425283 ]], dtype=float32)

In [82]:
print(f' มิติของ predictions2_1 =  {predictions2_1.ndim} และ มิติของ testY2_1 =  {testY2_1.ndim}')

 มิติของ predictions =  2 และ มิติของ testY =  2


In [83]:
predictions2_1ver2 = []
for i in tqdm(predictions2_1):
    if i > 0.5:
        predictions2_1ver2.append(1)
    else:
        predictions2_1ver2.append(0)
predictions2_1ver2 = np.array(predictions2_1ver2, dtype="float32")

predictions2_1ver2.shape = (len(predictions2_1ver2) ,1)                         #แปลงให้ predictions2 เป็น 2 มิติ 
predictions2_1ver2.ndim

100%|██████████| 37/37 [00:00<00:00, 47112.70it/s]


2

In [84]:
predictions2_1ver2[:5]

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [85]:
predictions2_1ver2.max(axis=1)

array([0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1.], dtype=float32)

In [86]:
testY2_1.max(axis=1)

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1])

In [87]:
print(classification_report(testY2_1.max(axis=1),	
                            predictions2_1ver2.max(axis=1),  
							target_names=le.classes_ ))

              precision    recall  f1-score   support

      Normal       1.00      0.56      0.71        18
   Pneumonia       0.70      1.00      0.83        19

    accuracy                           0.78        37
   macro avg       0.85      0.78      0.77        37
weighted avg       0.85      0.78      0.77        37



In [102]:
confusion_matrix(testY2_1.max(axis=1),	
                        predictions2_1ver2.max(axis=1) )

array([[10,  8],
       [ 0, 19]])

## 2) output 2 มิติ : ค่า predict คือ ความน่าจะเป็น 2 ค่า 
(ความน่าจะเป็นที่จะเป็นคลาส 0 และ 1)

In [88]:
data2_2 = np.array(train_img, dtype="float32") / 255.0

In [89]:
labels2_2 = np.array(data_train2[['Type1']] )         

In [90]:
le2_2 = LabelEncoder()                                                                   #จะแปลงชื่อ class เป็นตัวเลข
labels2_2 = le2_2.fit_transform(labels2_2)                                                  # dimention ของ Y จะเป็น 1 (37,) 
labels2_2 = to_categorical(labels2_2, num_classes=2)                            # แปลงเป็นเลข ดัมมี่ (1เป็น 1,0 ----- 0 เป็น  0,1)    
            #shape (148, 2) ค่าเดิม 0. กับ 1.  หลังจากรันจะเป็น [0., 1.] กับ [1., 0.]  ------->  แต่จะรัน model2.fit ไม่ได้ อาจเพราะเป็นโมเดลสำหรับ output แค่ 1 ค่า 

(trainX2_2, testX2_2, trainY2_2, testY2_2) = train_test_split(data2_2, labels2_2,
	                                                            test_size=0.25, random_state=40)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [91]:
le2_2.classes_

array(['Normal', 'Pneumonia'], dtype=object)

### train2_2

In [92]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [93]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [94]:
base_model2_2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [95]:
base_model2_2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2_2.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # ลองแก้ unit จาก 1 เป็น 2
model2_2 = tf.keras.models.Model(inputs=base_model2_2.input, outputs=prediction_layer)

In [96]:
model2_2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [105]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    mode="max"
)

In [106]:
H2_2 = model2_2.fit(
	x=aug.flow(trainX2_2, trainY2_2, batch_size=BS),
	validation_data=(testX2_2, testY2_2),
	steps_per_epoch=len(trainX2_2) // BS,
	epochs=10, callbacks=[callback])

Epoch 1/10
13/13 [==============================] - 29s 2s/step - loss: 0.6796 - accuracy: 0.5825 - val_loss: 0.6775 - val_accuracy: 0.5946
Epoch 2/10
13/13 [==============================] - 30s 2s/step - loss: 0.6519 - accuracy: 0.6505 - val_loss: 0.6590 - val_accuracy: 0.7027
Epoch 3/10
13/13 [==============================] - 29s 2s/step - loss: 0.6606 - accuracy: 0.6796 - val_loss: 0.6394 - val_accuracy: 0.7568
Epoch 4/10
13/13 [==============================] - 30s 2s/step - loss: 0.6264 - accuracy: 0.7308 - val_loss: 0.6222 - val_accuracy: 0.7838
Epoch 5/10
13/13 [==============================] - 30s 2s/step - loss: 0.5967 - accuracy: 0.7379 - val_loss: 0.6051 - val_accuracy: 0.7838
Epoch 6/10
13/13 [==============================] - 30s 2s/step - loss: 0.5704 - accuracy: 0.8447 - val_loss: 0.5889 - val_accuracy: 0.7838


### test

In [107]:
predictions2_2 = model2_2.predict(testX2_2, batch_size=BS)

In [108]:
predictions2_2.ndim

2

In [109]:
predictions2_2[:5]

array([[0.50359267, 0.38177377],
       [0.6646601 , 0.5416252 ],
       [0.4895034 , 0.6363899 ],
       [0.57706285, 0.64657414],
       [0.50352424, 0.65097564]], dtype=float32)

In [110]:
print(classification_report(testY2_2.argmax(axis=1),	
                            predictions2_2.argmax(axis=1),  
							target_names=le2_2.classes_ ))

              precision    recall  f1-score   support

      Normal       0.73      0.89      0.80        18
   Pneumonia       0.87      0.68      0.76        19

    accuracy                           0.78        37
   macro avg       0.80      0.79      0.78        37
weighted avg       0.80      0.78      0.78        37



In [111]:
confusion_matrix(testY2_2.argmax(axis=1),	
                        predictions2_2.argmax(axis=1) )

array([[16,  2],
       [ 6, 13]])

## Fine tune

In [ ]:
len(base_model2.layers)

707

In [ ]:
base_model2.trainable = True
for layer in base_model2.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#train
model2.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator,
                    callbacks=[callback2])

# 3 class ==>  ***DenseNet201***

###แบ่งข้อมูล3

In [118]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcancer])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [119]:
train_img3 = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img3.append(img)


100%|██████████| 180/180 [00:06<00:00, 26.56it/s]


In [120]:
data3 = np.array(train_img3, dtype="float32") / 255.0

In [121]:
labels3 = np.array(data_train3[['Type1']] )           #shape (180, 1) ค่าเป็น ชื่อคลาส Normal, Pneumonia , Lungcancer

In [122]:
le3 = LabelEncoder()                                                                     
labels3 = le3.fit_transform(labels3)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 
labels3 = to_categorical(labels3, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [123]:
(trainX3, testX3, trainY3, testY3) = train_test_split(data3, labels3,
	                                        test_size=0.25, random_state=52)

In [124]:
le3.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

## train3

In [125]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [126]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [127]:
base_model3 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [128]:
base_model3.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3 = tf.keras.models.Model(inputs=base_model3.input, outputs=prediction_layer)

In [129]:
model3.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [130]:
H3 = model3.fit(
	x=aug.flow(trainX3, trainY3, batch_size=BS),
	validation_data=(testX3, testY3),
	steps_per_epoch=len(trainX3) // BS,
	epochs=10)

Epoch 1/10
16/16 [==============================] - 48s 2s/step - loss: 0.6125 - accuracy: 0.5433 - val_loss: 0.6234 - val_accuracy: 0.4222
Epoch 2/10
16/16 [==============================] - 34s 2s/step - loss: 0.5952 - accuracy: 0.5669 - val_loss: 0.5982 - val_accuracy: 0.4444
Epoch 3/10
16/16 [==============================] - 34s 2s/step - loss: 0.5757 - accuracy: 0.5591 - val_loss: 0.5744 - val_accuracy: 0.5111
Epoch 4/10
16/16 [==============================] - 33s 2s/step - loss: 0.5472 - accuracy: 0.5827 - val_loss: 0.5528 - val_accuracy: 0.5333
Epoch 5/10
16/16 [==============================] - 34s 2s/step - loss: 0.5281 - accuracy: 0.7008 - val_loss: 0.5316 - val_accuracy: 0.6000
Epoch 6/10
16/16 [==============================] - 34s 2s/step - loss: 0.4959 - accuracy: 0.7165 - val_loss: 0.5125 - val_accuracy: 0.6667
Epoch 7/10
16/16 [==============================] - 33s 2s/step - loss: 0.4790 - accuracy: 0.7480 - val_loss: 0.4928 - val_accuracy: 0.6889
Epoch 8/10
16/16 [==

## test3

In [131]:
predictions3 = model3.predict(testX3, batch_size=BS)

In [132]:
predictions3[:10]

array([[0.11222601, 0.61711353, 0.41817784],
       [0.28418848, 0.06384382, 0.73389995],
       [0.09955165, 0.8149698 , 0.30933672],
       [0.29784667, 0.17570794, 0.6432164 ],
       [0.20061097, 0.35959214, 0.48942998],
       [0.10427403, 0.33945054, 0.39991271],
       [0.13755232, 0.41933763, 0.6521852 ],
       [0.1107538 , 0.4179282 , 0.24421173],
       [0.21606854, 0.54657173, 0.18179992],
       [0.09908774, 0.24719673, 0.61512876]], dtype=float32)

In [133]:
testY3.argmax(axis=1)

array([1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1,
       2])

In [134]:
print(classification_report(testY3.argmax(axis=1),	
                           predictions3.argmax(axis=1),  
                            target_names=le3.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      0.50      0.67         8
      Normal       0.85      0.94      0.89        18
   Pneumonia       0.81      0.89      0.85        19

    accuracy                           0.84        45
   macro avg       0.89      0.78      0.80        45
weighted avg       0.86      0.84      0.84        45



In [135]:
confusion_matrix(testY3.argmax(axis=1),	
                        predictions3_2.argmax(axis=1) )

array([[ 7,  0,  1],
       [ 0, 18,  0],
       [ 0,  2, 17]])

# 3 class ==> ***InceptionV3***

##แบ่งข้อมูล3

In [137]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcancer])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [143]:
train_img3_Incep = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (299, 299))                                   #ขนาดของ input
    train_img3_Incep.append(img)


100%|██████████| 180/180 [00:06<00:00, 26.71it/s]


In [144]:
data3_Incep = np.array(train_img3_Incep, dtype="float32") / 255.0

In [146]:
labels3_Incep = np.array(data_train3[['Type1']] )           #shape (180, 1) ค่าเป็น ชื่อคลาส Normal, Pneumonia , Lungcancer

In [147]:
le3_Incep = LabelEncoder()                                                                     
labels3_Incep = le3_Incep.fit_transform(labels3_Incep)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 --> ['Normal', 'Pneumonia', 'Lungcancer']
labels3_Incep = to_categorical(labels3_Incep, num_classes=3)                   #shape (180, 3)  มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [148]:
(trainX3_Incep, testX3_Incep, trainY3_Incep, testY3_Incep) = train_test_split(data3_Incep, labels3_Incep,
	                                        											test_size=0.25, random_state=52)

In [149]:
le3_Incep.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

## train3

In [150]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [151]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

InceptionV3

In [153]:
base_model3_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")

In [154]:
base_model3_Incep.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3_Incep = tf.keras.models.Model(inputs=base_model3_Incep.input, outputs=prediction_layer)

In [155]:
model3_Incep.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [160]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    mode="max"
)

In [161]:
H3_Incep = model3_Incep.fit(
	x=aug.flow(trainX3_Incep, trainY3_Incep, batch_size=BS),
	validation_data=(testX3_Incep, testY3_Incep),
	steps_per_epoch=len(trainX3_Incep) // BS,
	epochs=20, 
	callbacks = [callback])

Epoch 1/20
16/16 [==============================] - 33s 2s/step - loss: 0.2336 - accuracy: 0.9134 - val_loss: 0.2191 - val_accuracy: 1.0000
Epoch 2/20
16/16 [==============================] - 33s 2s/step - loss: 0.2179 - accuracy: 0.9449 - val_loss: 0.2163 - val_accuracy: 1.0000
Epoch 3/20
16/16 [==============================] - 33s 2s/step - loss: 0.2220 - accuracy: 0.9291 - val_loss: 0.2132 - val_accuracy: 1.0000


## test3

In [162]:
predictions3_Incep = model3_Incep.predict(testX3_Incep, batch_size=BS)

In [163]:
predictions3_Incep[:10]

array([[0.03081182, 0.58541363, 0.50679356],
       [0.08051986, 0.03728512, 0.7079966 ],
       [0.0433327 , 0.83353853, 0.2266548 ],
       [0.1410583 , 0.07304928, 0.74109054],
       [0.00798771, 0.28370184, 0.83665633],
       [0.06980368, 0.19123742, 0.621834  ],
       [0.06501719, 0.2103219 , 0.6884623 ],
       [0.0188987 , 0.81560916, 0.18352625],
       [0.00847757, 0.9069102 , 0.101549  ],
       [0.06826645, 0.18518716, 0.6713476 ]], dtype=float32)

In [164]:
testY3_Incep.argmax(axis=1)

array([1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1,
       2])

In [165]:
print(classification_report(testY3_Incep.argmax(axis=1),	
                           predictions3_Incep.argmax(axis=1),  
                            target_names=le3_Incep.classes_))

              precision    recall  f1-score   support

  Lungcancer       1.00      1.00      1.00         8
      Normal       1.00      1.00      1.00        18
   Pneumonia       1.00      1.00      1.00        19

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [166]:
le3_Incep.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

In [167]:
confusion_matrix(testY3_Incep.argmax(axis=1),	
                        predictions3_Incep.argmax(axis=1) )

array([[ 8,  0,  0],
       [ 0, 18,  0],
       [ 0,  0, 19]])